In [1]:
import billboard
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import base64
from requests import post, get
import json
import time
from datetime import datetime, timedelta


In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
data_df = pd.read_csv("data_df.csv")

# Display sample data
data_df.head()

,Song Name,Artist,Spotify ID,Genres,Duration,Popularity,Release Date
0,Bad Day,Daniel Powter,0mUyMawtxj1CJ76kn9gIZK,NaN,3:53,71,2005-02-22
1,Temperature,Sean Paul,0k2GOhqsrxDTAbFFSdNJjT,dancehall,3:38,78,2005-09-27
2,Promiscuous,"Nelly Furtado, Timbaland",2gam98EZKrF9XuOkU13ApN,NaN,4:02,83,2006-01-01
3,You're Beautiful,James Blunt,0vg4WnUWvze6pBOJDTq99k,NaN,3:29,73,2005-08-08
4,Hips Don't Lie (feat. Wyclef Jean),"Shakira, Wyclef Jean",3ZFTkvIE7kyPt6Nu3PEa7V,"latin pop, latin",3:38,84,2005-11-28


In [5]:
data_df["Year"] = ""
data_df.head()

,Song Name,Artist,Spotify ID,Genres,Duration,Popularity,Release Date,Year
0,Bad Day,Daniel Powter,0mUyMawtxj1CJ76kn9gIZK,NaN,3:53,71,2005-02-22,
1,Temperature,Sean Paul,0k2GOhqsrxDTAbFFSdNJjT,dancehall,3:38,78,2005-09-27,
2,Promiscuous,"Nelly Furtado, Timbaland",2gam98EZKrF9XuOkU13ApN,NaN,4:02,83,2006-01-01,
3,You're Beautiful,James Blunt,0vg4WnUWvze6pBOJDTq99k,NaN,3:29,73,2005-08-08,
4,Hips Don't Lie (feat. Wyclef Jean),"Shakira, Wyclef Jean",3ZFTkvIE7kyPt6Nu3PEa7V,"latin pop, latin",3:38,84,2005-11-28,


In [7]:
# Convert 'Year' column to numeric values, coercing errors to NaN
data_df['Year'] = pd.to_numeric(data_df['Year'], errors='coerce')

# Fill NaN values (if any) with 0 (or any other placeholder you want)
data_df['Year'].fillna(0, inplace=True)

# Convert 'Year' back to integers after filling NaN
data_df['Year'] = data_df['Year'].astype(int)

# Add 2006 to the 'Year' column for the first 100 rows
data_df.loc[:99, 'Year'] = 2006

# Add 1 to the 'Year' for every subsequent block of 100 rows (e.g., 2007, 2008, etc.)
for i in range(1, (len(data_df) // 100) + 1):
    start_idx = i * 100
    end_idx = (i + 1) * 100
    data_df.loc[start_idx:end_idx - 1, 'Year'] = 2006 + i

# Output the result
print(data_df[['Song Name', 'Year']].head(110))  # Display first 110 rows to check

                              Song Name  Year
0                               Bad Day  2006
1                           Temperature  2006
2                           Promiscuous  2006
3                      You're Beautiful  2006
4    Hips Don't Lie (feat. Wyclef Jean)  2006
..                                  ...   ...
105                    Before He Cheats  2007
106                   Hey There Delilah  2007
107                    I Wanna Love You  2007
108                        Say It Right  2007
109                           Glamorous  2007

[110 rows x 2 columns]


C:\Users\duvoe\AppData\Local\Temp\ipykernel_13904\2953739787.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data_df['Year'].fillna(0, inplace=True)


In [9]:
data_df.head(1900)

,Song Name,Artist,Spotify ID,Genres,Duration,Popularity,Release Date,Year
0,Bad Day,Daniel Powter,0mUyMawtxj1CJ76kn9gIZK,NaN,3:53,71,2005-02-22,2006
1,Temperature,Sean Paul,0k2GOhqsrxDTAbFFSdNJjT,dancehall,3:38,78,2005-09-27,2006
2,Promiscuous,"Nelly Furtado, Timbaland",2gam98EZKrF9XuOkU13ApN,NaN,4:02,83,2006-01-01,2006
3,You're Beautiful,James Blunt,0vg4WnUWvze6pBOJDTq99k,NaN,3:29,73,2005-08-08,2006
4,Hips Don't Lie (feat. Wyclef Jean),"Shakira, Wyclef Jean",3ZFTkvIE7kyPt6Nu3PEa7V,"latin pop, latin",3:38,84,2005-11-28,2006
...,...,...,...,...,...,...,...,...
1895,Bulletproof,La Roux,3kMrazSvILsgcwtidZd1Qd,NaN,3:25,71,2009,2024
1896,FE!N (feat. Playboi Carti),"Travis Scott, Playboi Carti",3KEMjT3QVfKIl4E9DIcrmF,rap,3:11,70,2023-07-26,2024
1897,The Painter,Cody Johnson,6UoKX6uLJwhsnyTp5k5StP,"texas country, country, red dirt",3:31,64,2023-08-11,2024
1898,Down Bad,Taylor Swift,5O4SPACkaBSKjBJEPYJwfQ,NaN,4:21,50,2024-04-19,2024


In [11]:
data_df.drop('Release Date', axis=1, inplace=True)
data_df.head()

,Song Name,Artist,Spotify ID,Genres,Duration,Popularity,Year
0,Bad Day,Daniel Powter,0mUyMawtxj1CJ76kn9gIZK,NaN,3:53,71,2006
1,Temperature,Sean Paul,0k2GOhqsrxDTAbFFSdNJjT,dancehall,3:38,78,2006
2,Promiscuous,"Nelly Furtado, Timbaland",2gam98EZKrF9XuOkU13ApN,NaN,4:02,83,2006
3,You're Beautiful,James Blunt,0vg4WnUWvze6pBOJDTq99k,NaN,3:29,73,2006
4,Hips Don't Lie (feat. Wyclef Jean),"Shakira, Wyclef Jean",3ZFTkvIE7kyPt6Nu3PEa7V,"latin pop, latin",3:38,84,2006


In [13]:
data_df['Rank'] = (data_df.index % 100) + 1
data_df.head(101)

,Song Name,Artist,Spotify ID,Genres,Duration,Popularity,Year,Rank
0,Bad Day,Daniel Powter,0mUyMawtxj1CJ76kn9gIZK,NaN,3:53,71,2006,1
1,Temperature,Sean Paul,0k2GOhqsrxDTAbFFSdNJjT,dancehall,3:38,78,2006,2
2,Promiscuous,"Nelly Furtado, Timbaland",2gam98EZKrF9XuOkU13ApN,NaN,4:02,83,2006,3
3,You're Beautiful,James Blunt,0vg4WnUWvze6pBOJDTq99k,NaN,3:29,73,2006,4
4,Hips Don't Lie (feat. Wyclef Jean),"Shakira, Wyclef Jean",3ZFTkvIE7kyPt6Nu3PEa7V,"latin pop, latin",3:38,84,2006,5
...,...,...,...,...,...,...,...,...
96,Feel Good Inc.,Gorillaz,0d28khcov6AiegSCpG5TuT,NaN,3:42,85,2006,97
97,I'm Sprung,T-Pain,0DLOyyQvwPTSDKuhpzMMwA,NaN,3:51,71,2006,98
98,Do I Make You Proud,Taylor Hicks,2jv9Ppwz9u3lqWfXV0q2sW,NaN,4:08,22,2006,99
99,For You I Will (Confidence),Kidz Bop Kids,43q9QXWgnB6b4qRq60CO0N,children's music,3:50,2,2006,100


In [15]:
data_df.rename(columns={'Song Name': 'Title'}, inplace=True)

In [17]:
new_df = data_df[['Year', 'Rank', 'Title', 'Artist', 'Spotify ID', 'Genres', 'Duration', 'Popularity']]
new_df.head(202)

,Year,Rank,Title,Artist,Spotify ID,Genres,Duration,Popularity
0,2006,1,Bad Day,Daniel Powter,0mUyMawtxj1CJ76kn9gIZK,NaN,3:53,71
1,2006,2,Temperature,Sean Paul,0k2GOhqsrxDTAbFFSdNJjT,dancehall,3:38,78
2,2006,3,Promiscuous,"Nelly Furtado, Timbaland",2gam98EZKrF9XuOkU13ApN,NaN,4:02,83
3,2006,4,You're Beautiful,James Blunt,0vg4WnUWvze6pBOJDTq99k,NaN,3:29,73
4,2006,5,Hips Don't Lie (feat. Wyclef Jean),"Shakira, Wyclef Jean",3ZFTkvIE7kyPt6Nu3PEa7V,"latin pop, latin",3:38,84
...,...,...,...,...,...,...,...,...
197,2007,98,Paralyzer,Finger Eleven,28IEbk5a7twNTbUEvWslUb,NaN,3:28,74
198,2007,99,That's That,lou deezi,08ZPPS44d64Vv7HnCy96a2,NaN,2:29,55
199,2007,100,Same Girl (with Usher),"R. Kelly, USHER",6OSmeoh8kXhCfz9WqvzDoJ,r&b,4:12,59
200,2008,1,Low (feat. T-Pain),"Flo Rida, T-Pain",0CAfXk7DXMnon4gLudAp7J,NaN,3:51,79


In [19]:
# Split 'Artist' column into multiple columns
artist_split = new_df['Artist'].str.split(', ', expand=True)

# Rename new columns as 'feat1', 'feat2', etc.
artist_split.columns = [f'feat{i+1}' for i in range(artist_split.shape[1])]

# Merge the new columns into the DataFrame
update_df = pd.concat([new_df, artist_split], axis=1)

update_df.head()

,Year,Rank,Title,Artist,Spotify ID,Genres,Duration,Popularity,feat1,feat2,feat3,feat4,feat5,feat6,feat7
0,2006,1,Bad Day,Daniel Powter,0mUyMawtxj1CJ76kn9gIZK,NaN,3:53,71,Daniel Powter,None,None,None,None,None,None
1,2006,2,Temperature,Sean Paul,0k2GOhqsrxDTAbFFSdNJjT,dancehall,3:38,78,Sean Paul,None,None,None,None,None,None
2,2006,3,Promiscuous,"Nelly Furtado, Timbaland",2gam98EZKrF9XuOkU13ApN,NaN,4:02,83,Nelly Furtado,Timbaland,None,None,None,None,None
3,2006,4,You're Beautiful,James Blunt,0vg4WnUWvze6pBOJDTq99k,NaN,3:29,73,James Blunt,None,None,None,None,None,None
4,2006,5,Hips Don't Lie (feat. Wyclef Jean),"Shakira, Wyclef Jean",3ZFTkvIE7kyPt6Nu3PEa7V,"latin pop, latin",3:38,84,Shakira,Wyclef Jean,None,None,None,None,None


In [21]:
artist_drop_df = update_df.drop(columns=['Artist'])
artist_drop_df.head()

,Year,Rank,Title,Spotify ID,Genres,Duration,Popularity,feat1,feat2,feat3,feat4,feat5,feat6,feat7
0,2006,1,Bad Day,0mUyMawtxj1CJ76kn9gIZK,NaN,3:53,71,Daniel Powter,None,None,None,None,None,None
1,2006,2,Temperature,0k2GOhqsrxDTAbFFSdNJjT,dancehall,3:38,78,Sean Paul,None,None,None,None,None,None
2,2006,3,Promiscuous,2gam98EZKrF9XuOkU13ApN,NaN,4:02,83,Nelly Furtado,Timbaland,None,None,None,None,None
3,2006,4,You're Beautiful,0vg4WnUWvze6pBOJDTq99k,NaN,3:29,73,James Blunt,None,None,None,None,None,None
4,2006,5,Hips Don't Lie (feat. Wyclef Jean),3ZFTkvIE7kyPt6Nu3PEa7V,"latin pop, latin",3:38,84,Shakira,Wyclef Jean,None,None,None,None,None


In [23]:
# Rename 'feat1' to 'Artist'
renamed_df = artist_drop_df.rename(columns={'feat1': 'Artist'})
renamed_df.head()

,Year,Rank,Title,Spotify ID,Genres,Duration,Popularity,Artist,feat2,feat3,feat4,feat5,feat6,feat7
0,2006,1,Bad Day,0mUyMawtxj1CJ76kn9gIZK,NaN,3:53,71,Daniel Powter,None,None,None,None,None,None
1,2006,2,Temperature,0k2GOhqsrxDTAbFFSdNJjT,dancehall,3:38,78,Sean Paul,None,None,None,None,None,None
2,2006,3,Promiscuous,2gam98EZKrF9XuOkU13ApN,NaN,4:02,83,Nelly Furtado,Timbaland,None,None,None,None,None
3,2006,4,You're Beautiful,0vg4WnUWvze6pBOJDTq99k,NaN,3:29,73,James Blunt,None,None,None,None,None,None
4,2006,5,Hips Don't Lie (feat. Wyclef Jean),3ZFTkvIE7kyPt6Nu3PEa7V,"latin pop, latin",3:38,84,Shakira,Wyclef Jean,None,None,None,None,None


In [29]:
# Rename 'feat1' to 'Artist'
renamed_df2 = renamed_df.rename(columns={'feat2': 'First Feature', 'feat3': 'Second Feature'})
renamed_df2.head(50)

,Year,Rank,Title,Spotify ID,Genres,Duration,Popularity,Artist,First Feature,Second Feature,feat4,feat5,feat6,feat7
0,2006,1,Bad Day,0mUyMawtxj1CJ76kn9gIZK,NaN,3:53,71,Daniel Powter,None,None,None,None,None,None
1,2006,2,Temperature,0k2GOhqsrxDTAbFFSdNJjT,dancehall,3:38,78,Sean Paul,None,None,None,None,None,None
2,2006,3,Promiscuous,2gam98EZKrF9XuOkU13ApN,NaN,4:02,83,Nelly Furtado,Timbaland,None,None,None,None,None
3,2006,4,You're Beautiful,0vg4WnUWvze6pBOJDTq99k,NaN,3:29,73,James Blunt,None,None,None,None,None,None
4,2006,5,Hips Don't Lie (feat. Wyclef Jean),3ZFTkvIE7kyPt6Nu3PEa7V,"latin pop, latin",3:38,84,Shakira,Wyclef Jean,None,None,None,None,None
5,2006,6,Unwritten,3U5JVgI2x4rDyHGObzJfNf,NaN,4:19,84,Natasha Bedingfield,None,None,None,None,None,None
6,2006,7,CRAZY,3AoEQRuFf8zVXWqSLo2UOi,k-pop,2:44,79,LE SSERAFIM,None,None,None,None,None,None
7,2006,8,Ridin',3kZoay4ANo86ehb6s4RwS9,southern hip hop,5:03,66,Chamillionaire,Krayzie Bone,None,None,None,None,None
8,2006,9,SexyBack (feat. Timbaland),0O45fw2L5vsWpdsOdXwNAR,NaN,4:02,79,Justin Timberlake,Timbaland,None,None,None,None,None
9,2006,10,Check On It (feat. Bun B & Slim Thug),2RWxrpsFshOBFwRBRstUlQ,NaN,3:30,58,Beyoncé,Bun B,Slim Thug,None,None,None,None


In [33]:
split_cleaned_df = renamed_df2[['Year', 'Rank', 'Title', 'Artist', 'First Feature', 'Second Feature', 'Genres', 'Duration', 'Popularity', 'Spotify ID']]
split_cleaned_df.head(202)

,Year,Rank,Title,Artist,First Feature,Second Feature,Genres,Duration,Popularity,Spotify ID
0,2006,1,Bad Day,Daniel Powter,None,None,NaN,3:53,71,0mUyMawtxj1CJ76kn9gIZK
1,2006,2,Temperature,Sean Paul,None,None,dancehall,3:38,78,0k2GOhqsrxDTAbFFSdNJjT
2,2006,3,Promiscuous,Nelly Furtado,Timbaland,None,NaN,4:02,83,2gam98EZKrF9XuOkU13ApN
3,2006,4,You're Beautiful,James Blunt,None,None,NaN,3:29,73,0vg4WnUWvze6pBOJDTq99k
4,2006,5,Hips Don't Lie (feat. Wyclef Jean),Shakira,Wyclef Jean,None,"latin pop, latin",3:38,84,3ZFTkvIE7kyPt6Nu3PEa7V
...,...,...,...,...,...,...,...,...,...,...
197,2007,98,Paralyzer,Finger Eleven,None,None,NaN,3:28,74,28IEbk5a7twNTbUEvWslUb
198,2007,99,That's That,lou deezi,None,None,NaN,2:29,55,08ZPPS44d64Vv7HnCy96a2
199,2007,100,Same Girl (with Usher),R. Kelly,USHER,None,r&b,4:12,59,6OSmeoh8kXhCfz9WqvzDoJ
200,2008,1,Low (feat. T-Pain),Flo Rida,T-Pain,None,NaN,3:51,79,0CAfXk7DXMnon4gLudAp7J


In [35]:
# Export the DataFrame to a JSON file
split_cleaned_df.to_json('split_cleaned_df.json', orient='records', lines=True)

In [ ]:
new_df.to_csv('songs_ranked.csv', index=False)

In [ ]:
nan_count = new_df['Genres'].isna().sum()

print(f"Number of NaN values in the 'Genres' column: {nan_count}")

In [ ]:
new_df.to_csv('songs_ranked.csv', index=False)